# Hackathon Dotz

##### Classificar as observações por (CATEGORIA | SUB-CATEGORIA)

In [1]:
# Importação dos pacotes para Analise

import pandas as pd
import seaborn as sns
import numpy as np
import matplotlib.pyplot as plt
import scipy.stats as sct
import statsmodels.api as sm
from google.colab import files  
import re



/usr/local/lib/python3.6/dist-packages/statsmodels/tools/_testing.py:19: FutureWarning: pandas.util.testing is deprecated. Use the functions in the public API at pandas.testing instead.
  import pandas.util.testing as tm


In [0]:
# Importando pacotes para treino e validação do modelo

from sklearn.feature_extraction.text import TfidfVectorizer
# um classificador linear que utiliza o Gradiente Descendente Estocástico como método de treino. 
# Por padrão, utiliza o estimador SVM.
from sklearn.linear_model import SGDClassifier
# Uma rede neural Perceptron Multicamadas
from sklearn.neural_network import MLPClassifier
from sklearn import metrics
from sklearn.metrics import confusion_matrix
from sklearn.model_selection import train_test_split



In [0]:
%matplotlib inline

In [4]:
#Carregar os dados para o google colab
uploaded = files.upload()

Saving Hackathon_Base_Teste.csv to Hackathon_Base_Teste.csv
Saving Hackathon_Base_Treino_comdep.csv to Hackathon_Base_Treino_comdep.csv


In [0]:
# Importação dos dataset

df = pd.read_csv('Hackathon_Base_Treino_comdep.csv')
df2 = pd.read_csv('Hackathon_Base_Teste.csv')


#### Classificando

In [0]:
# Criando a nova para classificar CATEGORIA

base = df

# Excluindo da descrição texto após os números, informações julgadas irrelevantes para a classificação.
df['DESC_AJUSTADA'] = df['DESCRIÇÃO PARCEIRO'].str.replace('[0-9].+', '', regex=True)

# Selecionando apenas o item a ser classificado e o target

base_data = base['DESC_AJUSTADA']
base_target = base['CATEGORIA']+' '+base['SUB-CATEGORIA']

In [0]:
# Transformando a descrição em vetor

stop_words=['de','da','do','com','para','c/','kg','un','ml','pct','gfa','jg','p/','la','&','.',',','-','+']
vetor = TfidfVectorizer(stop_words=stop_words, strip_accents='ascii')
vetor_x = vetor.fit_transform(base_data)


In [0]:
# Dividindo dataset em treino e teste

x_train, x_test, y_train, y_test = train_test_split(vetor_x, base_target, test_size=0.2, random_state=27)


In [10]:
# Aqui nós treinamos o classificador

clf = MLPClassifier(solver='lbfgs', alpha=1e-5, hidden_layer_sizes=(70, ), random_state=1, verbose=True, max_iter=200)
clf.fit(x_train, y_train)

/usr/local/lib/python3.6/dist-packages/sklearn/neural_network/_multilayer_perceptron.py:470: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
  self.n_iter_ = _check_optimize_result("lbfgs", opt_res, self.max_iter)


MLPClassifier(activation='relu', alpha=1e-05, batch_size='auto', beta_1=0.9,
              beta_2=0.999, early_stopping=False, epsilon=1e-08,
              hidden_layer_sizes=(70,), learning_rate='constant',
              learning_rate_init=0.001, max_fun=15000, max_iter=200,
              momentum=0.9, n_iter_no_change=10, nesterovs_momentum=True,
              power_t=0.5, random_state=1, shuffle=True, solver='lbfgs',
              tol=0.0001, validation_fraction=0.1, verbose=True,
              warm_start=False)

In [11]:
# Avaliando a performance com predição

predicted = clf.predict(x_test)

print('\n###Indicadores Classificação Categoria###\n')
print(metrics.classification_report(y_test, predicted))

# Matriz de confusão
print('\n### Matriz de Confusão Categoria###\n')
print(metrics.confusion_matrix(y_test, predicted))


###Indicadores Classificação Categoria###

                                                            precision    recall  f1-score   support

                               ABSORVENTE EXTERNO COM ABAS       0.73      1.00      0.84         8
                                ABSORVENTE EXTERNO NOTURNO       1.00      1.00      1.00         2
                        ABSORVENTE EXTERNO PROTETOR DIÁRIO       0.75      1.00      0.86         3
                               ABSORVENTE EXTERNO SEM ABAS       0.00      0.00      0.00         3
                                   ABSORVENTE INTERNO MINI       0.00      0.00      0.00         1
                                  ABSORVENTE INTERNO MÉDIO       0.00      0.00      0.00         0
                         ACESSÓRIOS DE BANHO BUCHA VEGETAL       0.00      0.00      0.00         1
                       ACESSÓRIOS DE BANHO ESCOVA DE BANHO       0.00      0.00      0.00         0
                      ACESSÓRIOS DE BANHO ESPONJA DE BA

/usr/local/lib/python3.6/dist-packages/sklearn/metrics/_classification.py:1272: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.6/dist-packages/sklearn/metrics/_classification.py:1272: UndefinedMetricWarning: Recall and F-score are ill-defined and being set to 0.0 in labels with no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


In [0]:
# Predição em novos dados

df2['DESC_AJUSTADA'] = df2['DESCRIÇÃO PARCEIRO'].str.replace('[0-9].+', '', regex=True)
base_novos = df2['DESC_AJUSTADA']

# Tranformando descrição em vetor
vetor_n = vetor.transform(base_novos)

#Previsão dos novos dados
predicted_novos = clf.predict(vetor_n)


In [0]:
# Adicionando a coluna
df_cat_sub = df2[['DESCRIÇÃO PARCEIRO','DESC_AJUSTADA']]
df_cat_sub['CATEGORIA | SUB-CATEGORIA'] = predicted_novos

In [14]:
# Novos Dados de teste incluindo o departamento
df_cat_sub.head(5)



,DESCRIÇÃO PARCEIRO,DESC_AJUSTADA,CATEGORIA | SUB-CATEGORIA
0,SAL ROSA HIMALAIA C/ ALHO 500G,SAL ROSA HIMALAIA C/ ALHO,SAL SAL PARA CHURRASCO
1,JG BOLA NATAL ACASA C/17 DR/PR/BC R952,JG BOLA NATAL ACASA C/,ENFEITES DE NATAL BOLA DE NATAL
2,MIST PREP LIQ CHA VDE LINEA 56ML-CX LIMAO,MIST PREP LIQ CHA VDE LINEA,SAPONÁCEO PÓ
3,ALICATE CUT+ESPAT MUNDIAL SANDY AMARELO,ALICATE CUT+ESPAT MUNDIAL SANDY AMARELO,ACESSÓRIOS MÃO E PÉ ALICATE
4,MEIA PIZZA PEPPERONI UN,MEIA PIZZA PEPPERONI UN,PIZZA FRANGO


### Referências

https://medium.com/luisfredgs/classificando-textos-com-machine-learning-e054ca7bf4e0

https://www.alura.com.br/artigos/classificando-textos-com-python